# SOMMAIRE

# importation de la base et préprocessing

In [1]:
import pandas as pd
import numpy as np 
# import matplotlib as plt
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import decomposition
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import  RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.svm import LinearSVR
from sklearn.svm import SVR
from sklearn.ensemble import BaggingRegressor

from sklearn.dummy import DummyRegressor
import xgboost as xgb
import multiprocessing
from xgboost import XGBRegressor
from category_encoders import TargetEncoder
import category_encoders as ce
from category_encoders import QuantileEncoder
import time
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from julearn import run_cross_validation
from julearn.utils import configure_logging
from julearn.model_selection import StratifiedGroupsKFold
from julearn.scoring import register_scorer
import math
pd.set_option('display.max_rows', 25)

In [2]:
buildingnr= pd.read_csv("buildingnr.csv",decimal=".",index_col=0)


In [3]:
#création d'une classe pour segmenter le quantile encoder 
buildingnr['log_TotalGHGEmissions_class']= pd.qcut(buildingnr['log_TotalGHGEmissions'],4,labels=False)


In [4]:
#base issue de RFE 50 features 
base_ml = buildingnr[['log_TotalGHGEmissions_class','log_TotalGHGEmissions',
                     'YearsENERGYSTARCertified', 'ENERGYSTARScore',
 'Distribution Center_GFA',
 'Electricity%',
 'NaturalGas%',
 'LPU1%',
 'LPU2%',
 'Data Center_eui%',
 'College/University_GFA%',
 'Other_GFA%',
 'Restaurant_GFA',
 'Worship Facility_GFA',
 'Other_eui%',
 'Non-Refrigerated Warehouse_GFA%',
 'DecBuilt',
 'Office_eui%',
 'Distribution Center_GFA%',
 'Non-Refrigerated Warehouse_eui%',
 'BuildingType',
 'Other - Entertainment/Public Assembly_GFA',
 'Retail Store_GFA',
 'Non-Refrigerated Warehouse_GFA',
 'Manufacturing/Industrial Plant_GFA',
 'Office_GFA',
 'Parking_eui%',
 'Retail Store_GFA%',
 'log_PropertyGFATotal',
 'parking%',
 'LargestPropertyUseType',
 'PrimaryPropertyType',
 'log_NaturalGas%',
 'log_Electricity%',
 'Latitude',
 'Longitude',
 'YearBuilt',
 'Office_GFA%',
 'NumberofFloors',
 'PropertyGFATotal',
 'LargestUse',
 'PropertyGFABuilding(s)',
 #'ListOfAllPropertyUseTypes',
 'log_parking%',
 'Other_GFA',
 'SecondLargestPropertyUseType',
 'log_PropertyGFABuilding(s)',
 'SecondLargestPropertyUseTypeGFA',
 'log_LargestPropertyUseTypeGFA',
 'log_SecondLargestPropertyUseTypeGFA',
 'ThirdLargestPropertyUseType',
 'LargestPropertyUseTypeGFA',
 'log_ThirdLargestPropertyUseTypeGFA']]



In [5]:
Report = pd.DataFrame(columns=['méthode', 'MSE','RMSE','R2 Test',"MAE",'R2 Train','Durée'])
#initialisation tableau de reporting 

In [6]:
colonnes = base_ml.select_dtypes(include= 'number').columns.tolist()
for col in colonnes : 
    mask = base_ml[col].isna()
    base_ml.loc[mask,col]= 0 
#remplacement des valeurs manquantes numériques par des 0 car il s'agit principalement des données surface des 2 et 3 usages
    
colonnes = base_ml.select_dtypes(include= 'object').columns.tolist()
for col in colonnes : 
    mask = base_ml[col].isna()
    base_ml.loc[mask,col]= 'missing'

    
X = base_ml.drop(columns = ['log_TotalGHGEmissions_class','log_TotalGHGEmissions'])
y = base_ml['log_TotalGHGEmissions']


colQE= base_ml.select_dtypes(include= 'object').columns.tolist()
colQE.append('DecBuilt') 

enc = QuantileEncoder(cols=colQE, quantile=0.5, m=1.0).fit(X, y)
X_QE= enc.transform(X) #transformation des feature catégorielle en numérique

X=X_QE

X.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
YearsENERGYSTARCertified,1634.0,3.904047,0.173103,2.420584,3.873934,3.873934,3.873934,5.453026e+00
ENERGYSTARScore,1634.0,43.208078,38.522158,0.000000,0.000000,47.000000,81.000000,1.000000e+02
Distribution Center_GFA,1634.0,1981.695838,13386.650609,0.000000,0.000000,0.000000,0.000000,2.880480e+05
Electricity%,1634.0,0.694516,0.264504,-0.336762,0.484575,0.694491,0.998805,1.000003e+00
NaturalGas%,1634.0,0.279909,0.263912,0.000000,0.000000,0.246140,0.494383,1.000000e+00
...,...,...,...,...,...,...,...,...
log_LargestPropertyUseTypeGFA,1634.0,10.830597,1.109126,0.000000,10.146640,10.681102,11.429367,1.604769e+01
log_SecondLargestPropertyUseTypeGFA,1634.0,4.569500,4.926300,0.000000,0.000000,0.000000,9.462109,1.336912e+01
ThirdLargestPropertyUseType,1634.0,3.929555,0.290735,2.891270,3.840801,3.840801,3.840801,5.999848e+00
LargestPropertyUseTypeGFA,1634.0,98638.148103,277961.039377,0.000000,25505.250000,43525.500000,91983.750000,9.320156e+06


In [7]:
nsplit = 5 #utilisé par StratifiedKfold

# Random Forest 

In [8]:
#Random Forest 
debut = time.time()

estimator = RandomForestRegressor(random_state=22, n_estimators = 2000)

numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])  

reg = Pipeline(steps=[('preprocessor', numeric_transformer),
                      ('regression', estimator) ])

# Create KFold object.

skf = StratifiedKFold(n_splits=nsplit, shuffle=True,random_state=22 ) 


for train_index, test_index in skf.split(X, base_ml['log_TotalGHGEmissions_class']):
    x_train, x_test = X.iloc[train_index] , X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    reg.fit(x_train, y_train)

        
    
Append= pd.DataFrame([['RandomForest' , 
                        np.around (mean_squared_error(y_test,reg.predict(x_test)),3),
                       np.around (mean_squared_error(y_test,reg.predict(x_test),squared=False),3),
                      np.around(r2_score(y_test,reg.predict(x_test)),3),
                     np.around( mean_absolute_error(y_test,reg.predict(x_test)),3),
                      np.around(r2_score(y_train,reg.predict(x_train)),3) ,
                      time.time() - debut]], 
                     columns=['méthode', 'MSE','RMSE','R2 Test','MAE','R2 Train','Durée'])
    

Report = pd.concat([Report, Append]) 
Report

,méthode,MSE,RMSE,R2 Test,MAE,R2 Train,Durée
0,RandomForest,0.422,0.649,0.801,0.448,0.976,269.627753


# LinearRegression

In [9]:
### LinearRegression
debut = time.time()
Fold_result = pd.DataFrame(columns=['méthode', 'MSE','RMSE','R2 Test',"MA%E",'R2 Train','Durée'])


reg = Pipeline(steps=[   ('preprocessor', numeric_transformer), ('regression', linear_model.LinearRegression())])
  


skf = StratifiedKFold(n_splits=nsplit, shuffle=True,random_state=22 ) 


for train_index, test_index in skf.split(X, base_ml['log_TotalGHGEmissions_class']):
    x_train, x_test = X.iloc[train_index] , X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]

reg.fit(x_train, y_train)
    
Append = pd.DataFrame([['LinearRegression' , 
                        np.around (mean_squared_error(y_test,reg.predict(x_test)),3),
                       np.around (mean_squared_error(y_test,reg.predict(x_test),squared=False),3),
                      np.around(r2_score(y_test,reg.predict(x_test)),3),
                     np.around( mean_absolute_error(y_test,reg.predict(x_test)),3),
                      np.around(r2_score(y_train,reg.predict(x_train)),3) ,
                      time.time() - debut]], 
                     columns=['méthode', 'MSE','RMSE','R2 Test','MAE','R2 Train','Durée'])
    


Report = pd.concat([Report, Append]) 
Report


,méthode,MSE,RMSE,R2 Test,MAE,R2 Train,Durée
0,RandomForest,0.422,0.649,0.801,0.448,0.976,269.627753
0,LinearRegression,0.534,0.731,0.748,0.488,0.83,0.050864


In [10]:
Report

,méthode,MSE,RMSE,R2 Test,MAE,R2 Train,Durée
0,RandomForest,0.422,0.649,0.801,0.448,0.976,269.627753
0,LinearRegression,0.534,0.731,0.748,0.488,0.83,0.050864


# Dummy 

In [11]:
debut = time.time()
#Dummy regressor 

reg = Pipeline(steps=[('preprocessor', numeric_transformer),
                      ('regression', DummyRegressor())])


skf = StratifiedKFold(n_splits=nsplit, shuffle=True,random_state=22 ) 


for train_index, test_index in skf.split(X, base_ml['log_TotalGHGEmissions_class']):
    x_train, x_test = X.iloc[train_index] , X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    reg.fit(x_train, y_train)

        
    
Append= pd.DataFrame([['Dummy' , 
                        np.around (mean_squared_error(y_test,reg.predict(x_test)),3),
                       np.around (mean_squared_error(y_test,reg.predict(x_test),squared=False),3),
                      np.around(r2_score(y_test,reg.predict(x_test)),3),
                     np.around( mean_absolute_error(y_test,reg.predict(x_test)),3),
                      np.around(r2_score(y_train,reg.predict(x_train)),3) ,
                      time.time() - debut]], 
                     columns=['méthode', 'MSE','RMSE','R2 Test','MAE','R2 Train','Durée'])
            
Report = pd.concat([Report, Append]) 


In [12]:
Report

,méthode,MSE,RMSE,R2 Test,MAE,R2 Train,Durée
0,RandomForest,0.422,0.649,0.801,0.448,0.976,269.627753
0,LinearRegression,0.534,0.731,0.748,0.488,0.83,0.050864
0,Dummy,2.122,1.457,-0.0,1.128,0.0,0.069815


# Ridge

In [13]:
#Ridge
debut = time.time()
reg = Pipeline(steps=[('preprocessor', numeric_transformer),
                      ('regression', linear_model.Ridge(random_state=22 ))])


skf = StratifiedKFold(n_splits=nsplit, shuffle=True,random_state=22 ) 


for train_index, test_index in skf.split(X, base_ml['log_TotalGHGEmissions_class']):
    x_train, x_test = X.iloc[train_index] , X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    reg.fit(x_train, y_train)

        
    
Append= pd.DataFrame([['Ridge' , 
                        np.around (mean_squared_error(y_test,reg.predict(x_test)),3),
                       np.around (mean_squared_error(y_test,reg.predict(x_test),squared=False),3),
                      np.around(r2_score(y_test,reg.predict(x_test)),3),
                     np.around( mean_absolute_error(y_test,reg.predict(x_test)),3),
                      np.around(r2_score(y_train,reg.predict(x_train)),3) ,
                      time.time() - debut]], 
                     columns=['méthode', 'MSE','RMSE','R2 Test','MAE','R2 Train','Durée'])
            
Report = pd.concat([Report, Append]) 


In [14]:
Report

,méthode,MSE,RMSE,R2 Test,MAE,R2 Train,Durée
0,RandomForest,0.422,0.649,0.801,0.448,0.976,269.627753
0,LinearRegression,0.534,0.731,0.748,0.488,0.83,0.050864
0,Dummy,2.122,1.457,-0.0,1.128,0.0,0.069815
0,Ridge,0.533,0.73,0.749,0.487,0.83,0.08179


# Ridge CV

In [15]:
#RidgeCV

debut = time.time()
reg = Pipeline(steps=[('preprocessor', numeric_transformer),
                      ('regression', linear_model.RidgeCV())])


skf = StratifiedKFold(n_splits=nsplit, shuffle=True,random_state=22 ) 


for train_index, test_index in skf.split(X, base_ml['log_TotalGHGEmissions_class']):
    x_train, x_test = X.iloc[train_index] , X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    reg.fit(x_train, y_train)

        
    
Append= pd.DataFrame([['RidgeCV' , 
                        np.around (mean_squared_error(y_test,reg.predict(x_test)),3),
                       np.around (mean_squared_error(y_test,reg.predict(x_test),squared=False),3),
                      np.around(r2_score(y_test,reg.predict(x_test)),3),
                     np.around( mean_absolute_error(y_test,reg.predict(x_test)),3),
                      np.around(r2_score(y_train,reg.predict(x_train)),3) ,
                      time.time() - debut]], 
                     columns=['méthode', 'MSE','RMSE','R2 Test','MAE','R2 Train','Durée'])
            
Report = pd.concat([Report, Append]) 


In [16]:
Report

,méthode,MSE,RMSE,R2 Test,MAE,R2 Train,Durée
0,RandomForest,0.422,0.649,0.801,0.448,0.976,269.627753
0,LinearRegression,0.534,0.731,0.748,0.488,0.83,0.050864
0,Dummy,2.122,1.457,-0.0,1.128,0.0,0.069815
0,Ridge,0.533,0.73,0.749,0.487,0.83,0.08179
0,RidgeCV,0.527,0.726,0.751,0.486,0.83,0.120678


# Lasso CV

In [ ]:
debut = time.time()
parameters = {'alpha': [1, 10, 100,1000,10000], 'tol': [1,0.1,0.01,0.001,0.0001, 0.00001, 0.000001],'selection':['random','cyclic']}
reg = Pipeline(steps=[('preprocessor', numeric_transformer),
                      ('regression', GridSearchCV(linear_model.Lasso(max_iter=10000, random_state=22),parameters))]) 


skf = StratifiedKFold(n_splits=nsplit, shuffle=True,random_state=22 ) 


for train_index, test_index in skf.split(X, base_ml['log_TotalGHGEmissions_class']):
    x_train, x_test = X.iloc[train_index] , X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    reg.fit(x_train, y_train)

        
    
Append= pd.DataFrame([['Lasso + GS_CV' , 
                        np.around (mean_squared_error(y_test,reg.predict(x_test)),3),
                       np.around (mean_squared_error(y_test,reg.predict(x_test),squared=False),3),
                      np.around(r2_score(y_test,reg.predict(x_test)),3),
                     np.around( mean_absolute_error(y_test,reg.predict(x_test)),3),
                      np.around(r2_score(y_train,reg.predict(x_train)),3) ,
                      time.time() - debut]], 
                     columns=['méthode', 'MSE','RMSE','R2 Test','MAE','R2 Train','Durée'])
            
Report = pd.concat([Report, Append]) 



In [ ]:
Report

,méthode,MSE,RMSE,R2 Test,MAE,R2 Train,Durée
0,RandomForest,0.422,0.649,0.801,0.448,0.976,269.627753
0,LinearRegression,0.534,0.731,0.748,0.488,0.83,0.050864
0,Dummy,2.122,1.457,-0.0,1.128,0.0,0.069815
0,Ridge,0.533,0.73,0.749,0.487,0.83,0.08179
0,RidgeCV,0.527,0.726,0.751,0.486,0.83,0.120678
0,Lasso + GS_CV,2.122,1.457,-0.0,1.128,0.0,6.654823


In [ ]:
Report

,méthode,MSE,RMSE,R2 Test,MAE,R2 Train,Durée
0,RandomForest,0.422,0.649,0.801,0.448,0.976,269.627753
0,LinearRegression,0.534,0.731,0.748,0.488,0.83,0.050864
0,Dummy,2.122,1.457,-0.0,1.128,0.0,0.069815
0,Ridge,0.533,0.73,0.749,0.487,0.83,0.08179
0,RidgeCV,0.527,0.726,0.751,0.486,0.83,0.120678
0,Lasso + GS_CV,2.122,1.457,-0.0,1.128,0.0,6.654823


# Random Forest GS-CV

In [20]:
#Random Forest CV 
debut = time.time()

tuned_parameters = { 'max_depth': [None, 1, 2, 3], 'min_samples_split': [2,5,10, 50, 100]}
#test sur 5000 estimators max voir temps 750 sec quasi pas d'amélioration) 
reg = Pipeline(steps=[('preprocessor', numeric_transformer),
                      ('regression', GridSearchCV(RandomForestRegressor(random_state=22), tuned_parameters, cv=5, n_jobs=-4, verbose=1))]) #possible de rajouter l'indicateur de scoring : scoring=r2_score par ex. 

# Create KFold object.

skf = StratifiedKFold(n_splits=nsplit, shuffle=True,random_state=22 ) 


for train_index, test_index in skf.split(X, base_ml['log_TotalGHGEmissions_class']):
    x_train, x_test = X.iloc[train_index] , X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    reg.fit(x_train, y_train)

        
    
Append= pd.DataFrame([['RandomForest CV' , 
                        np.around (mean_squared_error(y_test,reg.predict(x_test)),3),
                       np.around (mean_squared_error(y_test,reg.predict(x_test),squared=False),3),
                      np.around(r2_score(y_test,reg.predict(x_test)),3),
                     np.around( mean_absolute_error(y_test,reg.predict(x_test)),3),
                      np.around(r2_score(y_train,reg.predict(x_train)),3) ,
                      time.time() - debut]], 
                     columns=['méthode', 'MSE','RMSE','R2 Test','MAE','R2 Train','Durée'])
    





Report = pd.concat([Report, Append]) 


Fitting 5 folds for each of 20 candidates, totalling 100 fits
Fitting 5 folds for each of 20 candidates, totalling 100 fits
Fitting 5 folds for each of 20 candidates, totalling 100 fits
Fitting 5 folds for each of 20 candidates, totalling 100 fits
Fitting 5 folds for each of 20 candidates, totalling 100 fits


# XGBoost GS-CV

In [21]:
debut = time.time()


parameters = {'n_estimators': [500,1000], #[8000,10000] - R2 0.365
    'max_depth': [2,4],
    'gamma':[0.1,0.2], # Minimum loss reduction to create new tree split ,0.5,0.9
    'subsample':[0.7], 
    'reg_lambda':[0.1], 
    'reg_alpha':[0.1,0.8],
    'n_jobs':[-1],
     'booster':['gbtree']
              }
              
    

reg = Pipeline(steps=[('preprocessor', numeric_transformer),
                      ('regression', GridSearchCV(xgb.XGBRegressor(learning_rate=0.01,objective='reg:squarederror', 
                        random_state = 22) , parameters,scoring = 'r2',cv=5,verbose =1))])

skf = StratifiedKFold(n_splits=nsplit, shuffle=True,random_state=22 ) 


for train_index, test_index in skf.split(X, base_ml['log_TotalGHGEmissions_class']):
    x_train, x_test = X.iloc[train_index] , X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    reg.fit(x_train, y_train)

        
    
Append= pd.DataFrame([['XGB GS_CV' , 
                        np.around (mean_squared_error(y_test,reg.predict(x_test)),3),
                       np.around (mean_squared_error(y_test,reg.predict(x_test),squared=False),3),
                      np.around(r2_score(y_test,reg.predict(x_test)),3),
                     np.around( mean_absolute_error(y_test,reg.predict(x_test)),3),
                      np.around(r2_score(y_train,reg.predict(x_train)),3) ,
                      time.time() - debut]], 
                     columns=['méthode', 'MSE','RMSE','R2 Test','MAE','R2 Train','Durée'])
            
Report = pd.concat([Report, Append]) 
Report

Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits


,méthode,MSE,RMSE,R2 Test,MAE,R2 Train,Durée
0,RandomForest,0.422,0.649,0.801,0.448,0.976,269.627753
0,LinearRegression,0.534,0.731,0.748,0.488,0.83,0.050864
0,Dummy,2.122,1.457,-0.0,1.128,0.0,0.069815
0,Ridge,0.533,0.73,0.749,0.487,0.83,0.08179
0,RidgeCV,0.527,0.726,0.751,0.486,0.83,0.120678
0,Lasso + GS_CV,2.122,1.457,-0.0,1.128,0.0,6.654823
0,RandomForest CV,0.434,0.659,0.796,0.453,0.968,403.501971
0,XGB GS_CV,0.318,0.564,0.85,0.386,0.955,897.745335


# ElasticNet GS-CV

In [ ]:
#ElasticNet 
debut = time.time()

parameters = {'alpha': [1, 10, 100,1000,10000], 'l1_ratio':[0,0.25,0.75,1,100],'tol': [1,0.1,0.01,0.001,0.0001, 0.00001, 0.000001],
              'selection':['random','cyclic']}


reg = Pipeline(steps=[('preprocessor', numeric_transformer),
                      ('regression', GridSearchCV(ElasticNet(random_state=22,max_iter=1000),parameters))])

skf = StratifiedKFold(n_splits=nsplit, shuffle=True,random_state=22 ) 


for train_index, test_index in skf.split(X, base_ml['log_TotalGHGEmissions_class']):
    x_train, x_test = X.iloc[train_index] , X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    reg.fit(x_train, y_train)

        
    
Append= pd.DataFrame([['ElasticNet GS_CV' , 
                        np.around (mean_squared_error(y_test,reg.predict(x_test)),3),
                       np.around (mean_squared_error(y_test,reg.predict(x_test),squared=False),3),
                      np.around(r2_score(y_test,reg.predict(x_test)),3),
                     np.around( mean_absolute_error(y_test,reg.predict(x_test)),3),
                      np.around(r2_score(y_train,reg.predict(x_train)),3) ,
                      time.time() - debut]], 
                     columns=['méthode', 'MSE','RMSE','R2 Test','MAE','R2 Train','Durée'])
            
Report = pd.concat([Report, Append]) 


In [23]:
Report

,méthode,MSE,RMSE,R2 Test,MAE,R2 Train,Durée
0,RandomForest,0.422,0.649,0.801,0.448,0.976,269.627753
0,LinearRegression,0.534,0.731,0.748,0.488,0.83,0.050864
0,Dummy,2.122,1.457,-0.0,1.128,0.0,0.069815
0,Ridge,0.533,0.73,0.749,0.487,0.83,0.08179
0,RidgeCV,0.527,0.726,0.751,0.486,0.83,0.120678
0,Lasso + GS_CV,2.122,1.457,-0.0,1.128,0.0,6.654823
0,RandomForest CV,0.434,0.659,0.796,0.453,0.968,403.501971
0,XGB GS_CV,0.318,0.564,0.85,0.386,0.955,897.745335
0,ElasticNet GS_CV,0.624,0.79,0.706,0.585,0.737,120.580611


# ElasticNetCV

In [24]:
### ElasticNet 
debut = time.time()
reg = Pipeline(steps=[('preprocessor', numeric_transformer),
                      ('regression', linear_model.ElasticNetCV(random_state=22))])

skf = StratifiedKFold(n_splits=nsplit, shuffle=True,random_state=22 ) 


for train_index, test_index in skf.split(X, base_ml['log_TotalGHGEmissions_class']):
    x_train, x_test = X.iloc[train_index] , X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    reg.fit(x_train, y_train)

        
    
Append= pd.DataFrame([['ElasticCV' , 
                        np.around (mean_squared_error(y_test,reg.predict(x_test)),3),
                       np.around (mean_squared_error(y_test,reg.predict(x_test),squared=False),3),
                      np.around(r2_score(y_test,reg.predict(x_test)),3),
                     np.around( mean_absolute_error(y_test,reg.predict(x_test)),3),
                      np.around(r2_score(y_train,reg.predict(x_train)),3) ,
                      time.time() - debut]], 
                     columns=['méthode', 'MSE','RMSE','R2 Test','MAE','R2 Train','Durée'])
            
Report = pd.concat([Report, Append]) 

In [25]:
Report

,méthode,MSE,RMSE,R2 Test,MAE,R2 Train,Durée
0,RandomForest,0.422,0.649,0.801,0.448,0.976,269.627753
0,LinearRegression,0.534,0.731,0.748,0.488,0.83,0.050864
0,Dummy,2.122,1.457,-0.0,1.128,0.0,0.069815
0,Ridge,0.533,0.73,0.749,0.487,0.83,0.08179
0,RidgeCV,0.527,0.726,0.751,0.486,0.83,0.120678
0,Lasso + GS_CV,2.122,1.457,-0.0,1.128,0.0,6.654823
0,RandomForest CV,0.434,0.659,0.796,0.453,0.968,403.501971
0,XGB GS_CV,0.318,0.564,0.85,0.386,0.955,897.745335
0,ElasticNet GS_CV,0.624,0.79,0.706,0.585,0.737,120.580611
0,ElasticCV,0.523,0.723,0.754,0.485,0.82,0.619298


# Bagging

In [26]:
debut = time.time()
#tuned_parameters = {'n_estimators': [50, 100, 200], 'max_depth': [None, 1, 2, 3], 'min_samples_split': [10, 50, 100]}
#test sur 5000 estimators max voir temps 750 sec quasi pas d'amélioration) 
#X, y = make_regression(n_samples=100, n_features=4, n_informative=2, n_targets=1,random_state=22, shuffle=False)

reg = Pipeline(steps=[('preprocessor', numeric_transformer),                    
                      ('regression', BaggingRegressor(base_estimator=SVR(),n_estimators=10, random_state=22))])

skf = StratifiedKFold(n_splits=nsplit, shuffle=True,random_state=22 ) 


for train_index, test_index in skf.split(X, base_ml['log_TotalGHGEmissions_class']):
    x_train, x_test = X.iloc[train_index] , X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    reg.fit(x_train, y_train)

        
    
Append= pd.DataFrame([['BaggingRegressor' , 
                        np.around (mean_squared_error(y_test,reg.predict(x_test)),3),
                       np.around (mean_squared_error(y_test,reg.predict(x_test),squared=False),3),
                      np.around(r2_score(y_test,reg.predict(x_test)),3),
                     np.around( mean_absolute_error(y_test,reg.predict(x_test)),3),
                      np.around(r2_score(y_train,reg.predict(x_train)),3) ,
                      time.time() - debut]], 
                     columns=['méthode', 'MSE','RMSE','R2 Test','MAE','R2 Train','Durée'])
            
Report = pd.concat([Report, Append]) 

In [27]:
Report

,méthode,MSE,RMSE,R2 Test,MAE,R2 Train,Durée
0,RandomForest,0.422,0.649,0.801,0.448,0.976,269.627753
0,LinearRegression,0.534,0.731,0.748,0.488,0.83,0.050864
0,Dummy,2.122,1.457,-0.0,1.128,0.0,0.069815
0,Ridge,0.533,0.73,0.749,0.487,0.83,0.08179
0,RidgeCV,0.527,0.726,0.751,0.486,0.83,0.120678
0,Lasso + GS_CV,2.122,1.457,-0.0,1.128,0.0,6.654823
0,RandomForest CV,0.434,0.659,0.796,0.453,0.968,403.501971
0,XGB GS_CV,0.318,0.564,0.85,0.386,0.955,897.745335
0,ElasticNet GS_CV,0.624,0.79,0.706,0.585,0.737,120.580611
0,ElasticCV,0.523,0.723,0.754,0.485,0.82,0.619298


# LinearSVR-GSCV

In [ ]:
debut = time.time()
parameters = {'epsilon': [0], 'tol': [0.0001],'C':[1],'loss':["epsilon_insensitive","squared_epsilon_insensitive"],'intercept_scaling':[1],
              'dual':[True, False],'verbose':[0],'random_state':[22],'max_iter':[1000]}

reg = Pipeline(steps=[('preprocessor', numeric_transformer),
                      ('regression', GridSearchCV(LinearSVR(),parameters))]) 


skf = StratifiedKFold(n_splits=nsplit, shuffle=True,random_state=22 ) 


for train_index, test_index in skf.split(X, base_ml['log_TotalGHGEmissions_class']):
    x_train, x_test = X.iloc[train_index] , X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    reg.fit(x_train, y_train)

        
    
Append= pd.DataFrame([['Linear SVR GS_CV' , 
                        np.around (mean_squared_error(y_test,reg.predict(x_test)),3),
                       np.around (mean_squared_error(y_test,reg.predict(x_test),squared=False),3),
                      np.around(r2_score(y_test,reg.predict(x_test)),3),
                     np.around( mean_absolute_error(y_test,reg.predict(x_test)),3),
                      np.around(r2_score(y_train,reg.predict(x_train)),3) ,
                      time.time() - debut]], 
                     columns=['méthode', 'MSE','RMSE','R2 Test','MAE','R2 Train','Durée'])
            
Report = pd.concat([Report, Append]) 

In [29]:
Report

,méthode,MSE,RMSE,R2 Test,MAE,R2 Train,Durée
0,RandomForest,0.422,0.649,0.801,0.448,0.976,269.627753
0,LinearRegression,0.534,0.731,0.748,0.488,0.83,0.050864
0,Dummy,2.122,1.457,-0.0,1.128,0.0,0.069815
0,Ridge,0.533,0.73,0.749,0.487,0.83,0.08179
0,RidgeCV,0.527,0.726,0.751,0.486,0.83,0.120678
0,Lasso + GS_CV,2.122,1.457,-0.0,1.128,0.0,6.654823
0,RandomForest CV,0.434,0.659,0.796,0.453,0.968,403.501971
0,XGB GS_CV,0.318,0.564,0.85,0.386,0.955,897.745335
0,ElasticNet GS_CV,0.624,0.79,0.706,0.585,0.737,120.580611
0,ElasticCV,0.523,0.723,0.754,0.485,0.82,0.619298
